# Financial Econometrics II: Project 1
### Jakub Laza and David Cerny

### Packages set-up

In [1]:
import Pkg
Pkg.activate(".")

  Activating project at `~/Desktop/FinEcox2/Financial-Ecox-Project1`


In [2]:
Pkg.instantiate()

In [3]:
Pkg.add("MarketData")

   Resolving package versions...
  No Changes to `~/Desktop/FinEcox2/Financial-Ecox-Project1/Project.toml`
  No Changes to `~/Desktop/FinEcox2/Financial-Ecox-Project1/Manifest.toml`


In [4]:
using MarketData

### Get data from Yahoo Finance (We are working with AutoZone, Inc.)

In [5]:
# Fetch SP500 historical data
data = MarketData.yahoo(:AZO)

8458×6 TimeArray{Float64, 2, Date, Matrix{Float64}} 1991-04-02 to 2024-10-28
┌────────────┬─────────┬─────────┬─────────┬─────────┬──────────┬──────────┐
│            │ Open    │ High    │ Low     │ Close   │ AdjClose │ Volume   │
├────────────┼─────────┼─────────┼─────────┼─────────┼──────────┼──────────┤
│ 1991-04-02 │   6.875 │   7.375 │  6.5625 │ 7.34375 │  7.34375 │ 9.1472e6 │
│ 1991-04-03 │ 7.28125 │ 7.34375 │   6.875 │  7.3125 │   7.3125 │ 3.4032e6 │
│ 1991-04-04 │ 7.28125 │ 7.28125 │ 7.09375 │ 7.21875 │  7.21875 │ 831200.0 │
│ 1991-04-05 │   7.125 │  7.1875 │ 7.09375 │   7.125 │    7.125 │ 457200.0 │
│ 1991-04-08 │  7.0625 │  7.1875 │   6.875 │   7.125 │    7.125 │ 608800.0 │
│ 1991-04-09 │   7.125 │    7.25 │   7.125 │  7.1875 │   7.1875 │ 472400.0 │
│ 1991-04-10 │   7.125 │     7.5 │   7.125 │     7.5 │      7.5 │ 781600.0 │
│ 1991-04-11 │    7.75 │ 8.28125 │    7.75 │  8.0625 │   8.0625 │ 1.3392e6 │
│ 1991-04-12 │   8.125 │    8.25 │  8.0625 │ 8.15625 │  8.15625 │ 468800.0 │

### Data Exploration